# Teradata PDCR Info Report Demo

This notebook demonstrates how to connect to a Teradata database using the `TeradataConnection` manager and retrieve PDCR info data from the `DBC.DBCInfoV` view using the `PDCRInfoReport` class.

## 1. Import Required Libraries

Import necessary libraries for database connection and data processing.

In [ ]:
import logging
import sys
from pathlib import Path
from sqlalchemy import text
import pandas as pd

# Add src to path for imports
sys.path.insert(0, str(Path.cwd()))

# Import the connection module
from src.connection import TeradataConnection, TeradataConnectionError

# Configure logging to see connection information
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

print("Libraries imported successfully!")

## 2. Establish Database Connection

Initialize the connection manager and list available environments.

In [ ]:
try:
    # Initialize the connection manager
    conn_manager = TeradataConnection()
    print("✓ Connection manager initialized successfully")
    
    # List available environments
    environments = conn_manager.list_environments()
    print(f"✓ Available environments: {environments}")
    
except TeradataConnectionError as e:
    print(f"✗ Connection Error: {e}")
    print("\nPlease ensure:")
    print("1. td_env.yaml file exists in the project root")
    print("2. Copy td_env.yaml.template to td_env.yaml")
    print("3. Update credentials for your test/prod environments")

## 5. Execute a Sample Query

Use the context manager to execute a query and retrieve data as a pandas DataFrame.

## 6a. Using PDCRInfoReport for System Information

The `PDCRInfoReport` class provides convenient access to Teradata system information from DBC.DBCInfoV.


In [ ]:
try:
    # Import the PDCRInfoReport class
    from src.reports import PDCRInfoReport
    
    # Initialize the report generator
    pdcr_report = PDCRInfoReport()
    print("✓ PDCRInfoReport initialized successfully")
    
    # Get PDCR info data from DBC.DBCInfoV
    info_df = pdcr_report.get_dbcinfo("test")
    
    print(f"\n✓ Retrieved PDCR info data: {len(info_df)} rows")
    print("\nFirst few rows:")
    print(info_df.head(10))
    
    # Show some statistics
    print(f"\nUnique InfoKey values: {info_df['InfoKey'].nunique()}")
    print("\nSample InfoKey values:")
    print(info_df['InfoKey'].unique()[:5])
    
    # Clean up
    pdcr_report.close()
    print("\n✓ PDCRInfoReport connections closed")
    
except Exception as e:
    print(f"✗ Error retrieving PDCR info: {e}")
    print("\nNote: This example requires a valid Teradata connection.")
    print("Ensure your td_env.yaml has correct credentials for the 'test' environment.")


## 7. Close the Connection

Properly clean up connections when done.

In [ ]:
try:
    # Close all connections
    if 'conn_manager' in locals():
        conn_manager.close_all()
        print("✓ All connections closed successfully")
    else:
        print("No connection manager to close")
        
except Exception as e:
    print(f"✗ Error closing connections: {e}")